In [1]:
import geopandas as gpd
import osmnx as ox
import matplotlib.pyplot as plt
import math

In [2]:
def get_gdf(file_name):
    """
    Input
    'file_name': file_name of dataframe
    
    Output
    GeoDataFrame
    """
    gdf = gpd.read_file(file_name)
    
    #verify if projection is representative
    assert gdf.crs == 'epsg:4326', "geometric data will not project to faithfully to long,lat"
    
    gdf.crs = f'epsg:102100'
    gdf = gdf.to_crs(epsg=4326)
    
    return gdf


def calculate_center(north, south, east, west):
    
    center_long = (east + west)/2
    center_lat = (north + south)/2
    center_calculated = (center_lat, center_long)
    
    return center_calculated

def calculate_distance(n:int):
    """I plugged the maximum coordinates from the dataframe in to https://www.nhc.noaa.gov/gccalc.shtml 
    and see that the distance between the the coordinate (30.27,84.69) and (30.68, 83.98) where both are (N,W) 
    is 82km. Therefore, if we want n=100 splits of the map, say, we should have a distance of 410m.
    """
    assert is_perfect_square(n), "Error! The chosen n will not split the map into even pieces"
    if 41000 % n == 0:
        return 41000/n
    else:
        print("Error! The chosen n will not split the map into even pieces")
        
def calculate_nsew_splits(bbox, n):
    north_max, south_max, east_max, west_max = bbox[3], bbox[1], bbox[2], bbox[0]
    delta_lat = (south_max - north_max)/n
    delta_long = (west_max - east_max)/n
    lats = [north_max]
    longs = [west_max]
    for index in range(n):
        lats = lats + [lats[index] + delta_lat]
        longs = longs + [longs[index] + delta_long]
    bboxes = list()
    for index in range(n):
        bboxes = bboxes + [(longs[index],lats[index+1],lats[index],longs[index+1])]
    return bboxes

def is_perfect_square(n):
    square_root = int(math.sqrt(n))
    return square_root * square_root == n

def map_splitter(distance:int, figsize:int, dpi:int, scatter:float, alpha:float, center:tuple, bbox:tuple, iteration,
                dataframe):
    """
    Parameter 'distance' controls the amount of nodes to include in the graph
    that are 'distance' away, in physical distance measured in meters, from the center of graph. The actual calculation
    is determined by a secondary argument called dist_type 
    which we will fix to be bbox. This selection retains only those nodes within a bounding
    box of the distance parameter. 
    
    figsize is the dimension of the image in int x int. This should morally be a tuple, but we can live with a single
    input
    
    dpi specifies quality of the map
    
    The scatter parameter is passed to the axis as a parameter for marker size.
    
    alpha specifies gradient of colors between 0 (transparent) and 1 (opaque)
    """
    
    assert figsize in {256, 512}, "Figure size 'figsize' must be either 256x256 or 512x512"
    assert (alpha <=1) and (alpha >=0), "alpha must be between 0 and 1"
    
    streets = ox.graph_from_point(center, dist=distance, network_type='drive')
    buildings = ox.geometries_from_point(center, tags={'building': True}, dist=distance)
    parks = ox.geometries_from_point(center, tags={'leisure': 'park'}, dist=distance)
    water_bodies = ox.geometries_from_point(center, tags={'natural': 'water'}, dist=distance)
    fig, ax = plt.subplots(figsize=(figsize, figsize))
    ox.plot_graph(streets, ax=ax, bgcolor='black', edge_color='gray', node_size=0, edge_linewidth=0.5, show=False)
    buildings.plot(ax=ax, color='yellow', alpha=alpha)  #Adjust colors as needed
    parks.plot(ax=ax, color='green', alpha=alpha)  # Adjust colors as needed
    water_bodies.plot(ax=ax, color='blue', alpha=alpha)  #Adjust colors as needed
    fig.tight_layout(pad=0)
    plt.savefig('tallahassee_base_{}.png'.format(iteration), dpi=dpi)
    fig, ax = plt.subplots(figsize=(figsize, figsize))
    ox.plot_graph(streets, ax=ax, bgcolor='white', edge_color='white', node_size=0, edge_linewidth=0.5, show=False)
    gdf = dataframe
    x = gdf.geometry.x
    y = gdf.geometry.y
    hb = ax.scatter(x, y, alpha=(alpha/4), c='black', s=scatter)  # Adjust color as needed
    fig.tight_layout(pad=0)
    plt.savefig('tallahassee_heat_{}.png'.format(iteration), dpi=dpi)

In [ ]:
from tqdm import tqdm

file_name = 'TOPS 365days Jan 3.geojson'
gdf = get_gdf(file_name)
bbox = gdf.total_bounds 
north, south, east, west = bbox[3], bbox[1], bbox[2], bbox[0]
n = 100
bboxes = calculate_nsew_splits(bbox, n)
distance = calculate_distance(n)
print("creating splits..")
for number, bbox in tqdm(enumerate(bboxes)):
    center = calculate_center(bbox[3], bbox[1], bbox[2], bbox[0])
    map_splitter(distance=distance, figsize=256, dpi=64, scatter=10, alpha=0.5, center=center, bbox=bbox, iteration=number,
                dataframe=gdf)

creating splits..


/gpfs/home/am18fh/gpfs/home/am18fh/.conda/envs/simplices/lib64/python3.6/site-packages/osmnx/utils_geo.py:335: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for polygon in geometry:
